In [1]:
import numpy as np
from utils_adaptive import run_adaptive
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('legend', fontsize=12)
matplotlib.rcParams['savefig.dpi'] = 1200

%load_ext autoreload
%autoreload 2
import time
import scipy.stats as scs
import math

from utils_carryover import theta1, theta2
from utils_adaptive import *
from utils_empirical import *

In [2]:
out_dir = "../result/flu-adaptive/"
out_path = out_dir 
save_path = out_dir + "flu_adaptive-final.pkl"

In [8]:
def generate_more_design(T):
    result = dict()
    half_T = int(T/4); more_T = int(T*2)
    
    for this_T in [5, 10, 15, 20, 25, 30, 40, 50]:
        if this_T < T:
            result[this_T] = [(2*t+1)/(2*this_T) for t in range(this_T)] + [1-1/(2*half_T)]*(T-this_T)
        else:
            result[this_T] = [(2*t+1)/(2*this_T) for t in range(T)]
            
            
    result['opt'] = [(2*t+1)/(2*T) for t in range(T)]


    out_df = pd.DataFrame(result)

    return out_df

In [5]:
save_path = out_dir + "flu_adaptive.pkl"
Y = import_MS_FLU_pos_data()
tau_ratio = -0.1

tau = np.mean(Y)*tau_ratio
pad = False

print_epochs = 10e4
var_thres = (tau/1.65)**2

summary = dict()

In [6]:
N = 50; num_mc= 500; num_iter=1; 
fs_pct = 0.2;
adj_N = N
kappa = var_thres

start_time = time.time()
for T_max in [21]:
# for T_max in [7,9,10,11,13,14,15,18,21,24,25,28,30,35,40,42]:
    
    np.random.seed(321)
    all_idx_N_list = list(); all_idx_T_list = list(); all_shuffle_list = list()
    for _ in range(num_iter):
        idx_N_list, idx_T_list, shuffle_list = sample_subblocks(Y, N, T_max, num_mc=num_mc)
        all_idx_N_list.append(idx_N_list); all_idx_T_list.append(idx_T_list); all_shuffle_list.append(shuffle_list)

    print(T_max)

    for j in range(num_iter):
        new_timestamp = time.time()
        print("iter: ", j, time.time()-start_time)
        timestamp = new_timestamp


        idx_N_list = all_idx_N_list[j]; idx_T_list = all_idx_T_list[j]; shuffle_list = all_shuffle_list[j]
        all_Ys = get_all_Ys(Y, idx_N_list, idx_T_list, shuffle_list)
     
        out = static_adaptive(tau, kappa, var_thres=var_thres, fs_pct=fs_pct, all_Ys=all_Ys,  
                              N=N, T_max=T_max, adaptive=True, adj_N=adj_N, print_out=False)

        
    out_df = np.array(out)
    print(np.median(out_df[:,:3] ** 2, axis=0))
    print(np.mean(out_df[:,:3] ** 2, axis=0))
    summary[T_max] = out
    with open(save_path, 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(time.time() - start_time)

7
iter:  0 0.015017986297607422
100/500 done
200/500 done
300/500 done
400/500 done
500/500 done
[1.03744820e-08 1.08570871e-08 1.15316286e-08]
[5.00753177e-08 4.44997742e-08 4.25343207e-08]
62.06526303291321


In [4]:
N = 50; num_mc= 500; num_iter=1; 
fs_pct = 0.2;
adj_N = N
kappa = var_thres

start_time = time.time()
for T_max in [7]:
# for T_max in [7,9,10,11,13,14,15,18,21,24,25,28,30,35,40,42]:
    
    np.random.seed(321)
    all_idx_N_list = list(); all_idx_T_list = list(); all_shuffle_list = list()
    for _ in range(num_iter):
        idx_N_list, idx_T_list, shuffle_list = sample_subblocks(Y, N, T_max, num_mc=num_mc)
        all_idx_N_list.append(idx_N_list); all_idx_T_list.append(idx_T_list); all_shuffle_list.append(shuffle_list)

    print(T_max)

    for j in range(num_iter):
        new_timestamp = time.time()
        print("iter: ", j, time.time()-start_time)
        timestamp = new_timestamp


        idx_N_list = all_idx_N_list[j]; idx_T_list = all_idx_T_list[j]; shuffle_list = all_shuffle_list[j]
        all_Ys = get_all_Ys(Y, idx_N_list, idx_T_list, shuffle_list)
     
        out = static_adaptive(tau, kappa, var_thres=var_thres, fs_pct=fs_pct, all_Ys=all_Ys,  
                              N=N, T_max=T_max, adaptive=True, adj_N=adj_N, print_out=False)

        
    out_df = np.array(out)
    print(np.median(out_df[:,:3] ** 2, axis=0))
    print(np.mean(out_df[:,:3] ** 2, axis=0))
    summary[T_max] = out
    with open(save_path, 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(time.time() - start_time)

7
iter:  0 0.014130830764770508


/Users/ruoxuanxiong/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


100/500 done
200/500 done
300/500 done
400/500 done
500/500 done
[9.23501147e-09 1.40752303e-08 1.31913640e-08]
[5.25117610e-08 4.95632784e-08 4.68498871e-08]
65.4692018032074


In [9]:
out_df[:,3]

array([3., 6., 4., 6., 6., 6., 3., 6., 6., 6., 6., 6., 6., 6., 3., 6., 6.,
       3., 6., 3., 6., 3., 6., 6., 6., 3., 3., 3., 6., 3., 5., 6., 6., 6.,
       6., 5., 6., 6., 6., 6., 6., 6., 6., 3., 6., 4., 6., 6., 3., 6., 3.,
       6., 6., 6., 6., 3., 6., 6., 6., 6., 3., 6., 3., 3., 6., 6., 6., 6.,
       3., 3., 6., 6., 6., 6., 6., 6., 3., 6., 6., 3., 3., 6., 4., 6., 6.,
       6., 6., 3., 6., 5., 6., 6., 6., 3., 6., 3., 3., 6., 3., 3., 6., 6.,
       6., 6., 6., 3., 6., 6., 6., 3., 6., 3., 6., 6., 6., 6., 6., 3., 6.,
       3., 3., 4., 6., 6., 3., 6., 6., 6., 6., 6., 6., 6., 6., 3., 6., 3.,
       6., 3., 6., 6., 3., 6., 6., 5., 6., 3., 3., 3., 6., 3., 3., 6., 6.,
       3., 6., 6., 6., 6., 6., 6., 6., 4., 6., 6., 6., 6., 6., 6., 3., 6.,
       6., 4., 6., 6., 6., 6., 3., 3., 6., 4., 3., 6., 6., 3., 6., 4., 3.,
       3., 6., 3., 6., 6., 3., 4., 3., 3., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 3., 6., 6., 6., 3., 6., 4., 3., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 5.